In [1]:
import pandas as pd
import nltk
import tensorflow as tf


In [2]:
data = pd.read_csv("JobID-53_Concordance_H3_to_S3.CSV",encoding="latin-1",dtype="object")

In [3]:
data.head()

,HS 2007 Product Code,HS 2007 Product Description,SITC Revision 3 Product Code,SITC Revision 3 Product Description
0,010110,Live horses/asses/mules/hinnies: pure-bred bre...,00151,Horses
1,010190,Live horses/asses/mules/hinnies other than pur...,00152,"Asses/mules/hinnies,live"
2,010210,Live bovine animals: pure-bred breeding animals,00111,"Bovine animals, breeding"
3,010290,Live bovine animals other than pure-bred breed...,00119,"Bovine animals, other"
4,010310,Live swine: pure-bred breeding animals,00131,"Swine, live for breeding"


In [4]:
instance = data[data.columns[0]][0]

In [5]:
[i for i in instance]

[u'0', u'1', u'0', u'1', u'1', u'0']

In [6]:
product_codes = data[data.columns[0]]
characters = []
for code in product_codes:
    characters.append([c for c in code])


In [7]:
len(characters)

5050

Prepare strings
-----

In [8]:
descriptions = data['HS 2007 Product Description'].str.replace("/"," ")

In [9]:
tokens = [nltk.word_tokenize(desc) for desc in descriptions]

In [10]:
def build_vocabulary(token_list):
    vocabulary = {}
    start_vocab = ["_PAD","_GO","_EOS","_UNK"]
    for row in token_list:
        for word in row:
            if word in vocabulary:
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1
    vocab = start_vocab + sorted(vocabulary,key=vocabulary.get,reverse=True)
    return vocab   

In [11]:
vocab = build_vocabulary(tokens)

In [12]:
dec_vocab = build_vocabulary(characters)

In [13]:
vocab

['_PAD',
 '_GO',
 '_EOS',
 '_UNK',
 u',',
 u'of',
 u'.',
 u'&',
 u'(',
 u')',
 u'other',
 u'not',
 u'excl',
 u'for',
 u'in',
 u'than',
 u'a',
 u'the',
 u'with',
 u'whether',
 u'>',
 u';',
 u'fibres',
 u'containing',
 u'by',
 u'incl',
 u'cotton',
 u'fabrics',
 u'weight',
 u'steel',
 u'n.e.s',
 u'similar',
 u'%',
 u'crocheted',
 u'more',
 u'but',
 u'knitted',
 u'machines',
 u'materials',
 u'<',
 u'articles',
 u'iron',
 u"'",
 u'metal',
 u'85',
 u'Woven',
 u'up',
 u'thereof',
 u'products',
 u'textile',
 u'yarn',
 u'used',
 u'kind',
 u"'s",
 u'rubber',
 u'Parts',
 u'fresh',
 u'sheets',
 u'hair',
 u'paper',
 u'put',
 u'further',
 u'on',
 u'synthetic',
 u'staple',
 u'animal',
 u'man-made',
 u'their',
 u'parts',
 u'wool',
 u'apparatus',
 u'width',
 u'forms',
 u'thread',
 u'worked',
 u'Other',
 u'salts',
 u'without',
 u'm2',
 u'weighing',
 u'solely',
 u'preparations',
 u'prepared',
 u'derivatives',
 u'sewing',
 u'frozen',
 u'retail',
 u'like',
 u'plastics',
 u'coated',
 u'sale',
 u'to',
 u'sin

In [14]:
dec_vocab

['_PAD',
 '_GO',
 '_EOS',
 '_UNK',
 u'0',
 u'1',
 u'2',
 u'9',
 u'3',
 u'4',
 u'8',
 u'5',
 u'6',
 u'7']

In [15]:
[ch.insert(0,"_GO") for ch in characters]
[ch.append("_EOS") for ch in characters]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [16]:
[token.append("_EOS") for token in tokens]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
len(characters)

5050

In [18]:
# Add padding to the description sequences
# Only padding input because HS codes are already fixed length
buckets = [(5,8),(10,8),(20,8),(30,8),(40,8),(50,8),(60,8),(70,8)]

In [45]:
import numpy as np
lengths = [len(token) for token in tokens]
np.mean(lengths)

21.141584158415842

In [20]:
def make_buckets(tokens, buckets):
    lengths = [len(token) for token in tokens]
    padding = []
    i = 0
    for length in lengths:
        for bucket in buckets:
            if length <= bucket[0]:
                padding.append((bucket[0]-length))
                pad(tokens[i],bucket[0]-length)
                break
        i = i+1
    return padding

def pad(sentence,padding_size):
    for i in range(0,padding_size):
        sentence.insert(0,"_PAD")

padding = make_buckets(tokens,buckets)

In [21]:
tokens

[[u'Live',
  u'horses',
  u'asses',
  u'mules',
  u'hinnies',
  u':',
  u'pure-bred',
  u'breeding',
  u'animals',
  '_EOS'],
 ['_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  u'Live',
  u'horses',
  u'asses',
  u'mules',
  u'hinnies',
  u'other',
  u'than',
  u'pure-bred',
  u'breeding',
  u'animals',
  '_EOS'],
 ['_PAD',
  '_PAD',
  u'Live',
  u'bovine',
  u'animals',
  u':',
  u'pure-bred',
  u'breeding',
  u'animals',
  '_EOS'],
 ['_PAD',
  u'Live',
  u'bovine',
  u'animals',
  u'other',
  u'than',
  u'pure-bred',
  u'breeding',
  u'animals',
  '_EOS'],
 ['_PAD',
  '_PAD',
  '_PAD',
  u'Live',
  u'swine',
  u':',
  u'pure-bred',
  u'breeding',
  u'animals',
  '_EOS'],
 ['_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  u'Live',
  u'swine',
  u'other',
  u'than',
  u'pure-bred',
  u'breeding',
  u'animals',
  u',',
  u'weighing',
  u'<',
  u'50kg',
  '_EOS'],
 ['_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '_PAD',
  '

In [22]:
tf.nn.seq2seq.basic_rnn_seq2seq

<function tensorflow.python.ops.seq2seq.basic_rnn_seq2seq>

In [23]:
tf.nn.seq2seq.model_with_buckets

<function tensorflow.python.ops.seq2seq.model_with_buckets>

In [27]:
word_id = dict(zip(vocab,[i for i in range(0,len(vocab))]))
code_id = dict(zip(dec_vocab,[i for i in range(0,len(dec_vocab))]))

Prepare Input Vectors for Model
---------

In [32]:
def make_inputs(instances,vocabulary):
    inputs = []
    for row in instances:
        inputs.append([vocabulary[token] for token in row])
    return inputs

encoder_inputs = make_inputs(tokens,word_id)
decoder_inputs = make_inputs(characters,code_id)

In [50]:
cell = tf.nn.rnn_cell.BasicRNNCell(15)

#tf.nn.seq2seq.basic_rnn_seq2seq(encoder_inputs,decoder_inputs,cell)
tf.nn.seq2seq.model_with_buckets(encoder_inputs,decoder_inputs, )

<function tensorflow.python.ops.seq2seq.model_with_buckets>